# Implementation of HMM algorithms

## Choosing the appropriate data structures
You can choose any data structure to represent your HMM as long as your implementation works. The important thing to bear in mind is that your model needs to contain: 
* N states  
  Represented by characters, strings or numbers. Can include the first and the last state explicitly but can also work without them. 
* A set of transition probabilities {a<sub>kj</sub>}  
  a<sub>kj</sub> = P(π(i) = j | π(i-1) = k)  
  The probability of being in the state j at step i given that at step i-1 we were in the state k.
* A set of emission probabilities {e<sub>k</sub>(c)}  
  e<sub>k</sub>(c) = P(s<sub>i</sub> = c | π(i) = k)  
  The probability of observing the symbol c at the i-th position of the sequence given that at the i-th step we are in the state k.
  
One possible representation is described below.

### States
States are represented as a list of characters.

In [ ]:
states = ['b','y','n','e']

### Transitions
Transitions are represented as a dictionary where keys are tuples and values are transition probabilities. The first element of the tuple is the state from which we transition and the second element is the state we transition to.

In [ ]:
transitions = {('b','y') : 0.2,
               ('b','n') : 0.8,
               ('y','y') : 0.7,
               ('y','n') : 0.2,
               ('y','e') : 0.1,
               ('n','n') : 0.8,
               ('n','y') : 0.1,
               ('n','e') : 0.1
    }

### Emissions
Emissions are represented as a dictionary where keys are states and values are dictionaries. The internal dictionaries contain emitted symbols as keys and emission probabilities as values.

In [ ]:
emissions = {'y' : {'A':0.1, 'C':0.4, 'G':0.4, 'T':0.1},
             'n' : {'A':0.25, 'C':0.25, 'G':0.25, 'T':0.25}
    }

### Sequence
The sequence is simply a string.

In [ ]:
sequence = 'ATGCG'

## Initializing matrices
We can write a simple utility function to initialize matrices that will be useful afterwards. It takes the desired number of rows, the desired number of columns and the value to fill the matrix with. If the third parameter is not provided, the matrix is filled with zeros by default.

In [ ]:
def initialize_matrix(dim1,dim2,value=0):
    F = []
    for i in range(0,dim1):
        F.append([])
        for j in range(0,dim2):
            F[i].append(value)
    return F

## Visualizing matrices
For visualization purposes only we implement some printing functions. They are not important for the sake of the solution and the details of the implementation can be ignored.

In [ ]:
def print_matrix(matrix,axis1,axis2):
    w = '{:<10}'
    print w.format('') + w.format('0') + ''.join([w.format(char) for char in axis2]) + w.format('0')
    for i, row in enumerate(matrix):
        print w.format(axis1[i]) + ''.join(['{:<10.2e}'.format(item) for item in row])
        
def print_matrix_p(matrix,axis1,axis2):
    w = '{:<10}'
    print w.format('') + w.format('0') + ''.join([w.format(char) for char in axis2]) + w.format('0')
    for i, row in enumerate(matrix):
        print w.format(axis1[i]) + ''.join(['{:<10s}'.format(item) for item in row])

## Forward algorithm
We start by initializing matrix F that will contain as many rows as there are states and as many columns as there are symbols in the sequence plus two additional columns (the first and the last). The probability of starting from the begin state is 1 so we set the first element of the matrix to 1. 

In [ ]:
F = initialize_matrix(len(states),len(sequence)+2)
F[0][0] = 1

print_matrix(F,states,sequence)

Next, we calculate the values for the first symbol. For each state, it is the probability of transitioning from the begin state to the current state times the probability of emitting the first symbol of the sequence in the current state.

In [ ]:
for i in range(1,len(states)-1):
        F[i][1] = transitions[(states[0],states[i])]*emissions[states[i]][sequence[0]]
        
print_matrix(F,states,sequence)

For all of the other symbols, from the second to the last, we calculate the values as the sum of probabilities. For each state, it is the sum of probabilities of transitioning from any previous state to the current state times the probability of emitting the corresponding symbol of the sequence in the current state times the probability of the previous state.

In [ ]:
for j in range(2,len(sequence)+1): # loops on the symbols
        for i in range(1,len(states)-1): # loops on the states
            p_sum = 0
            for k in range(1,len(states)-1): # loops on all of the possible previous states
                p_sum += F[k][j-1]*transitions[(states[k],states[i])]*emissions[states[i]][sequence[j-1]]
            F[i][j] = p_sum
            
print_matrix(F,states,sequence)

Now, we need to calculate the final value. It is the sum of probabilities of transitioning from any previous state to the end state times the probability of the previous state.

In [ ]:
p_sum = 0
for k in range(1,len(states)-1):
    p_sum += F[k][len(sequence)]*transitions[(states[k],states[-1])]
F[-1][-1] = p_sum

print_matrix(F,states,sequence)

We can now put everything into a function that takes states, transitions, emissions and a sequence and returns a matrix F.

In [ ]:
def forward(states,transitions,emissions,sequence):
    F = initialize_matrix(len(states),len(sequence)+2)
    F[0][0] = 1
    for i in range(1,len(states)-1):
        F[i][1] = transitions[(states[0],states[i])]*emissions[states[i]][sequence[0]]
    for j in range(2,len(sequence)+1):
        for i in range(1,len(states)-1):
            p_sum = 0
            for k in range(1,len(states)-1):
                p_sum += F[k][j-1]*transitions[(states[k],states[i])]*emissions[states[i]][sequence[j-1]]
            F[i][j] = p_sum
    p_sum = 0
    for k in range(1,len(states)-1):
        p_sum += F[k][len(sequence)]*transitions[(states[k],states[-1])]
    F[-1][-1] = p_sum
    return F

## Backward algorithm
We start by initializing matrix F that will contain as many rows as there are states and as many columns as there are symbols in the sequence plus two additional columns (the first and the last). The probability of starting from the end state is 1 so we set the last element of the matrix to 1.

In [ ]:
F = initialize_matrix(len(states),len(sequence)+2)
F[-1][-1] = 1

print_matrix(F,states,sequence)

Next, we calculate the values for the last symbol. For each state, it is the probability of transitioning from the current state to the end state.

In [ ]:
for i in range(1,len(states)-1):
    F[i][-2] = transitions[(states[i],states[-1])]
    
print_matrix(F,states,sequence)

For all of the other symbols, from the second to last to the first, we calculate the values as the sum of probabilities. For each state, it is the sum of probabilities of transitioning from the current state to any successive state times the probability of emitting the corresponding symbol of the sequence in the successive state times the probability of the successive state.

In [ ]:
for j in range(len(sequence)-1,0,-1): # loops on the symbols
    for i in range(1,len(states)-1): # loops on the states
        p_sum = 0
        for k in range(1,len(states)-1): # loops on all of the possible successive states
            p_sum += F[k][j+1]*transitions[(states[i],states[k])]*emissions[states[k]][sequence[j]]
        F[i][j] = p_sum

print_matrix(F,states,sequence)

Now, we need to calculate the final value. It is the sum of probabilities of transitioning from the begin state to any successive state times the probability of emitting the first symbol of the sequence in the successive state times the probability of the successive state.

In [ ]:
p_sum = 0
for k in range(1,len(states)-1):
    p_sum += F[k][1]*transitions[(states[0],states[k])]*emissions[states[k]][sequence[0]]
F[0][0] = p_sum

print_matrix(F,states,sequence)

We can now put everything into a function that takes states, transitions, emissions and a sequence and returns a matrix F.

In [ ]:
def backward(states,transitions,emissions,sequence):
    F = initialize_matrix(len(states),len(sequence)+2)
    F[-1][-1] = 1
    for i in range(1,len(states)-1):
        F[i][-2] = transitions[(states[i],states[-1])]*emissions[states[i]][sequence[-1]]
    for j in range(len(sequence)-1,0,-1):
        for i in range(1,len(states)-1):
            p_sum = 0
            for k in range(1,len(states)):
                p_sum += F[k][j+1]*transitions[(states[i],states[k])]*emissions[states[i]][sequence[j-1]]
            F[i][j] = p_sum
    p_sum = 0
    for k in range(1,len(states)-1):
        p_sum += F[k][1]*transitions[(states[0],states[k])]
    F[0][0] = p_sum
    return F

## Viterbi algorithm
We start by initializing matrices F and FP that will contain as many rows as there are states and as many columns as there are symbols in the sequence plus two additional columns (the first and the last). Matrix FP is filled with 'b' symbols that indicate the begin state. The probability of starting from the begin state is 1 so we set the first element of the matrix F to 1.

In [ ]:
F = initialize_matrix(len(states),len(sequence)+2)
FP = initialize_matrix(len(states),len(sequence)+2,states[0])
F[0][0] = 1

print_matrix(F,states,sequence)
print
print_matrix_p(FP,states,sequence)

Next, we calculate the values for the first symbol. For each state, it is the probability of transitioning from the begin state to the current state times the probability of emitting the first symbol of the sequence in the current state.

In [ ]:
for i in range(1,len(states)-1):
    F[i][1] = transitions[(states[0],states[i])]*emissions[states[i]][sequence[0]]
    
print_matrix(F,states,sequence)

In order to choose the maximum probability at each of the following steps we will need to find the maximum in a list of values. It will also be useful to know the index of the state that has this maximum probability so that we can set the corresponding pointer. Therefore, we implement a simple function that, given a list of values, returns the maximum value and the index of the maximum value.

In [ ]:
def get_max_val_ind(values):
    max_val = values[0]
    max_ind = 0
    for ind, val in enumerate(values):
        if val>max_val:
            max_val = val
            max_ind = ind
    return max_val, max_ind

For all of the other symbols, from the second to the last, we calculate the values as the maximum of probabilities. For each state, it is the maximum of probabilities of transitioning from any previous state to the current state times the probability of emitting the corresponding symbol of the sequence in the current state times the probability of the previous state. We also set the corresponding pointers accordingly.

In [ ]:
for j in range(2,len(sequence)+1): # loops on the symbols
        for i in range(1,len(states)-1): # loops on the states
            values = []
            for k in range(1,len(states)-1): # loops on all of the possible previous states
                values.append(F[k][j-1]*transitions[(states[k],states[i])]*emissions[states[i]][sequence[j-1]]) # appends the value to a list
            max_val, max_ind = get_max_val_ind(values) # finds the maximum and the index of the maximum in the list
            F[i][j] = max_val # sets the probability to the maximum probability
            FP[i][j] = states[max_ind+1] # sets the corresponding pointer to the appropriate previous state

print_matrix(F,states,sequence)
print
print_matrix_p(FP,states,sequence)

Now, we need to calculate the final value. It is the maximum of probabilities of transitioning from any previous state to the end state times the probability of the previous state.

In [ ]:
values = []
for k in range(1,len(states)-1):
    values.append(F[k][len(sequence)]*transitions[(states[k],states[-1])])
max_val, max_ind = get_max_val_ind(values)
F[-1][-1] = max_val
FP[-1][-1] = states[max_ind+1]

print_matrix(F,states,sequence)
print
print_matrix_p(FP,states,sequence)

We can now put everything into a function that takes states, transitions, emissions and a sequence and returns a matrix F and a matrix FP of pointers.

In [ ]:
def viterbi(states,transitions,emissions,sequence):
    F = initialize_matrix(len(states),len(sequence)+2)
    FP = initialize_matrix(len(states),len(sequence)+2,states[0])
    F[0][0] = 1
    for i in range(1,len(states)-1):
        F[i][1] = transitions[(states[0],states[i])]*emissions[states[i]][sequence[0]]
    for j in range(2,len(sequence)+1):
        for i in range(1,len(states)-1):
            values = []
            for k in range(1,len(states)-1):
                values.append(F[k][j-1]*transitions[(states[k],states[i])]*emissions[states[i]][sequence[j-1]])
            max_val, max_ind = get_max_val_ind(values)
            F[i][j] = max_val
            FP[i][j] = states[max_ind+1]
    values = []
    for k in range(1,len(states)-1):
        values.append(F[k][len(sequence)]*transitions[(states[k],states[-1])])
    max_val, max_ind = get_max_val_ind(values)
    F[-1][-1] = max_val
    FP[-1][-1] = states[max_ind+1]
    return F, FP

In order to traceback the path we need to start from the end state and move through the FP matrix according to the pointers. Therefore, we implement a function that takes states and the FP matrix, starts from the last element of the matrix and moves through the matrix appending, at each step, the current state to the path variable.

In [ ]:
def traceback(states,FP):
    path = ['e'] # the last element of the path is the end state
    current = FP[-1][-1] # the current state is the one written in the last cell of the matrix
    for i in range(len(FP[0])-2,0,-1): # loops on the symbols
        path = [current] + path # appends the current state to the path
        current = FP[states.index(current)][i] # finds the index of the current state in the list of states and moves to the corresponing row of FP 
    path = ['b'] + path # the first element of the path is the begin state
    return ' '.join(path) # transforms the list into a string where elements are separated by spaces

We can now visualize the most likely path we obtained.

In [ ]:
path = traceback(states,FP)
print '- '+' '.join(sequence)+' -'
print path